<a href="https://colab.research.google.com/github/amitbasuri/llm-finetune/blob/main/youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install transformers datasets nltk spacy sentence-transformers
%pip install -U sentence-transformers
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sent

In [1]:
import json
import nltk
import re
from transformers import pipeline
from nltk.tokenize import sent_tokenize
import spacy
from sentence_transformers import SentenceTransformer, util
from sklearn.model_selection import train_test_split
import pandas as pd


In [2]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from google.colab import files

uploaded = files.upload()


Saving [English (auto-generated)] How Developments In Nuclear Fusion Change Everything _ Neil deGrasse Tyson Explains... [DownSub.com].txt to [English (auto-generated)] How Developments In Nuclear Fusion Change Everything _ Neil deGrasse Tyson Explains... [DownSub.com].txt
Saving [English (auto-generated)] Is the Big Bang Theory Wrong_ _ Neil deGrasse Tyson Explains... [DownSub.com].txt to [English (auto-generated)] Is the Big Bang Theory Wrong_ _ Neil deGrasse Tyson Explains... [DownSub.com].txt
Saving [English (auto-generated)] Neil & NASA Explain James Webb's First Images [DownSub.com].txt to [English (auto-generated)] Neil & NASA Explain James Webb's First Images [DownSub.com].txt
Saving [English (auto-generated)] Neil deGrasse Tyson Explains Time Dilation [DownSub.com].txt to [English (auto-generated)] Neil deGrasse Tyson Explains Time Dilation [DownSub.com].txt
Saving [English (auto-generated)] What is Schrödinger's Cat_ _ Neil deGrasse Tyson Explains... [DownSub.com].txt to [En

In [5]:
def clean_transcript(transcript):
    # Remove timestamps (e.g., (00:00))
    transcript = re.sub(r'\(\d{2}:\d{2}\)', '', transcript)

    # Remove annotations within brackets (e.g., [points to slide])
    transcript = re.sub(r'\[.*?\]', '', transcript)

    # Remove any other unwanted characters or annotations
    transcript = re.sub(r'[^A-Za-z0-9.,!?;:\'\"\s]', '', transcript)

    # Normalize whitespace
    transcript = re.sub(r'\s+', ' ', transcript).strip()

    return transcript


In [6]:
import os

# List of uploaded files
transcript_files = [file for file in uploaded.keys() if file.endswith('.txt')]

cleaned_transcripts = {}

for file in transcript_files:
    with open(file, 'r', encoding='utf-8') as f:
        transcript = f.read()
    cleaned = clean_transcript(transcript)
    cleaned_transcripts[file] = cleaned

# Display cleaned transcript example
for file, content in cleaned_transcripts.items():
    print(f"--- Cleaned Transcript: {file} ---")
    print(content[:500])  # Print first 500 characters
    print("\n")


--- Cleaned Transcript: [English (auto-generated)] How Developments In Nuclear Fusion Change Everything _ Neil deGrasse Tyson Explains... [DownSub.com].txt ---
so this will be a benchmark day we'll look back on that fondly Chuck what's been bugging you lately I'll tell you I don't know if it's bugging me maybe I'm happy about it maybe maybe the entire human race has something to be hopeful about and you know how I feel about climate and the climate crisis right so they they I wake up this morning and it's all about fusion fusion everywhere everybody's talking about fusion fusion fusion I'm like whoa whoa is this a good thing or are they yanking my chai


--- Cleaned Transcript: [English (auto-generated)] Is the Big Bang Theory Wrong_ _ Neil deGrasse Tyson Explains... [DownSub.com].txt ---
hi chuck i got another explainer for you oh well i'm here to get it yeah yeah there's no end of these i don't think that's well no i will run out at one point my brain will be empty and i'll just be t

In [8]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [9]:
def segment_transcript(transcript, max_sentences=10):
    sentences = sent_tokenize(transcript)
    segments = []
    for i in range(0, len(sentences), max_sentences):
        segment = ' '.join(sentences[i:i+max_sentences])
        segments.append(segment)
    return segments

segmented_transcripts = {}

for file, content in cleaned_transcripts.items():
    segments = segment_transcript(content, max_sentences=10)
    segmented_transcripts[file] = segments

# Display segments example
for file, segments in segmented_transcripts.items():
    print(f"--- Segments for {file} ---")
    for i, segment in enumerate(segments[:2]):  # Show first 2 segments
        print(f"Segment {i+1}:")
        print(segment)
        print("\n")
    print("\n")


--- Segments for [English (auto-generated)] How Developments In Nuclear Fusion Change Everything _ Neil deGrasse Tyson Explains... [DownSub.com].txt ---
Segment 1:
so this will be a benchmark day we'll look back on that fondly Chuck what's been bugging you lately I'll tell you I don't know if it's bugging me maybe I'm happy about it maybe maybe the entire human race has something to be hopeful about and you know how I feel about climate and the climate crisis right so they they I wake up this morning and it's all about fusion fusion everywhere everybody's talking about fusion fusion fusion I'm like whoa whoa is this a good thing or are they yanking my chain are they just making me what are they doing are they are they doing the Lucy pull the football away from me what is happening what is happening with the fusion oh okay so uh the Highlight is at one of the National Laboratories uh Lawrence Livermore National Laboratories where they they among all the National Labs are one of their ma

In [10]:
# Initialize summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Initialize question generation pipeline
# We'll use a T5 model fine-tuned for question generation
qg_pipeline = pipeline("text2text-generation", model="valhalla/t5-base-qg-hl")

# Initialize question answering pipeline
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [11]:
def generate_summary(segment):
    try:
        summary = summarizer(segment, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
    except Exception as e:
        print(f"Summarization Error: {e}")
        summary = "Summary not available."
    return summary


In [12]:
def generate_questions(segment, num_questions=3):
    try:
        prompt = f"Generate {num_questions} relevant questions based on the following text:\n\n{segment}"
        generated = qg_pipeline(prompt, max_length=100, num_return_sequences=1)[0]['generated_text']
        questions = [q.strip() for q in generated.split('\n') if q.strip()]
        return questions[:num_questions]
    except Exception as e:
        print(f"Question Generation Error: {e}")
        return [f"Question {i+1} not available." for i in range(num_questions)]


In [13]:
def generate_answer(question, segment):
    try:
        answer = qa_pipeline(question=question, context=segment)['answer']
    except Exception as e:
        print(f"Question Answering Error: {e}")
        answer = "Answer not available."
    return answer


In [15]:
data = []

for file, segments in segmented_transcripts.items():
    for segment in segments:

        # Question-Answering Pairs
        questions = generate_questions(segment, num_questions=5)
        for question in questions:
            answer = generate_answer(question, segment)
            data.append({
                "instruction": "Answer the following question based on the lecture segment.",
                "input": question,
                "output": answer
            })

# Display some examples
for entry in data:
    print(json.dumps(entry, indent=4))
    print("\n")


{
    "instruction": "Answer the following question based on the lecture segment.",
    "input": "What is the name of the first fusion reactor?",
    "output": "hydrogen bombs"
}


{
    "instruction": "Answer the following question based on the lecture segment.",
    "input": "What is the name of the scientist who is a fan of science?",
    "output": "einstein"
}


{
    "instruction": "Answer the following question based on the lecture segment.",
    "input": "What is the name of the first astronomical project that i've been a part of?",
    "output": "zero wow"
}


{
    "instruction": "Answer the following question based on the lecture segment.",
    "input": "What is the name of the theory that Einstein formulated?",
    "output": "general theory of relativity"
}


{
    "instruction": "Answer the following question based on the lecture segment.",
    "input": "What is the schrodinger's cat's physics?",
    "output": "quantum physics"
}


{
    "instruction": "Answer the following